# Necessary Steps

In [87]:
import json
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil
from random import choice
import re
import csv
import zipfile
import pandas as pd
import glob
import math
import geopandas as gpd
from shapely.geometry import shape, Polygon, box
from PIL import Image
import requests
import time
from ultralytics import YOLO
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.util import img_as_ubyte
from shapely.ops import unary_union
from itertools import product
from samgeo import tms_to_geotiff
import pandas as pd

In [88]:
cd /home/core-stack/Documents/Ponds

/home/core-stack/Documents/Ponds


In [89]:
pwd = os.getcwd()
pwd

'/home/core-stack/Documents/Ponds'

In [90]:
import ee

In [91]:
ee.Authenticate()

True

In [92]:
ee.Initialize(project='ee-corestackdev')

# Paths and Args

In [93]:
# Zoom level 
zoom = 17

# Folder paths where you want to save image tiles
image_dir = "Data/Zoom17/TRY"

# Scale of image tile
scale = 1           # scale of 1 = 256*256 dimensional image

# load trained model
model_path = "Models/Ponds_best.pt"

# CSV file name where masks of detected object will be saved
csv_file = "CSV_Output/TRY.csv"

# Entropy threshold needed to calculate entropy (only in wet ponds case)
entropy_threshold = 2.5


# Download Data for Inference

In [94]:
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = (0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8))
    return int(round(pixel_y))

def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return int(round(pixel_x))

def pixel_x_to_lon(pixel_x, zoom):
    return (pixel_x / (2 ** (zoom + 8))) * 360 - 180

def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    return math.degrees(math.atan(math.sinh(n)))

def lat_lon_from_pixel(lat, lon, zoom, scale):
    tile_width_deg = pixel_x_to_lon(256, zoom) - pixel_x_to_lon(0, zoom)
    new_lon = lon + (tile_width_deg * scale)
    new_pixel_y = lat_to_pixel_y(lat, zoom) + (256 * scale)
    new_lat = pixel_y_to_lat(new_pixel_y, zoom)
    return new_lat, new_lon


def divide_tiff_into_chunks_with_mapping(chunk_size, output_dir, zoom, top_left_lat, top_left_lon, mapping_file="tile_mapping.csv"):
    """Splits a TIFF into chunks and saves a mapping of chunk names to tile names."""
    input_image_path = os.path.join(output_dir, 'field.tif')
    image = Image.open(input_image_path)
    width, height = image.size

    top_left_x_tile = int(lon_to_pixel_x(top_left_lon, zoom) / 256)
    top_left_y_tile = int(lat_to_pixel_y(top_left_lat, zoom) / 256)

    os.makedirs(os.path.join(output_dir, "chunks"), exist_ok=True)

    mappings = []

    for row, i in enumerate(range(0, width, chunk_size)):
        for col, j in enumerate(range(0, height, chunk_size)):
            box = (i, j, i + chunk_size, j + chunk_size)
            chunk = image.crop(box)

            chunk_filename = f'chunk_{row}_{col}.tif'
            chunk.save(os.path.join(output_dir, "chunks", chunk_filename))

            x_tile = top_left_x_tile + (i // chunk_size)
            y_tile = top_left_y_tile + (j // chunk_size)
            tile_filename = f'tile_{zoom}_{x_tile}_{y_tile}.tif'

            mappings.append([chunk_filename, tile_filename])

    print("Image has been split into chunks correctly.")

    # Save mapping to a CSV file
    mapping_path = os.path.join(output_dir, mapping_file)
    with open(mapping_path, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Chunk Name", "Tile Name"])
        writer.writerows(mappings)

    print(f"Mapping file saved: {mapping_path}")


            
def download(coords, output_dir, zoom, scale):
    """Improved download function that handles both points and bboxes"""
    # Check if input is a single point (lat, lon) or bbox [min_lat, min_lon, max_lat, max_lon]
    if len(coords) == 2:  # It's a single point
        lat, lon = coords
        # Create a bbox around the point using the scale
        new_lat, new_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        bbox = [min(lat, new_lat), min(lon, new_lon), max(lat, new_lat), max(lon, new_lon)]
    else:  # It's already a bbox
        bbox = coords
    
    # Rest of your download function remains the same
    min_lon = min(bbox[1], bbox[3])
    max_lon = max(bbox[1], bbox[3])
    min_lat = min(bbox[0], bbox[2])
    max_lat = max(bbox[0], bbox[2])
    
    # Calculate the exact tile boundaries (fixed missing parentheses)
    tile_x_min = math.floor(lon_to_pixel_x(min_lon, zoom) / 256)
    tile_x_max = math.ceil(lon_to_pixel_x(max_lon, zoom) / 256)
    tile_y_min = math.floor(lat_to_pixel_y(max_lat, zoom) / 256)  # Note: y tiles increase downward
    tile_y_max = math.ceil(lat_to_pixel_y(min_lat, zoom) / 256)
    
    # Calculate the exact geographic bounds of the tile area
    exact_min_lon = pixel_x_to_lon(tile_x_min * 256, zoom)
    exact_max_lon = pixel_x_to_lon(tile_x_max * 256, zoom)
    exact_min_lat = pixel_y_to_lat(tile_y_max * 256, zoom)
    exact_max_lat = pixel_y_to_lat(tile_y_min * 256, zoom)
    
    corrected_bbox = [exact_min_lon, exact_min_lat, exact_max_lon, exact_max_lat]
    print(f"Downloading tiles for exact bbox: {corrected_bbox}")
    
    os.makedirs(os.path.join(output_dir, "chunks"), exist_ok=True)
    tms_to_geotiff(output=os.path.join(output_dir, "field.tif"), 
                  bbox=corrected_bbox, 
                  zoom=zoom, 
                  source='Satellite', 
                  overwrite=True)
    
    # Modify the divide function to use exact coordinates
    divide_tiff_into_chunks_with_mapping(256, output_dir, zoom, exact_max_lat, exact_min_lon)


                  
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon)]
    for _ in range(n):
        pixel_x = lon_to_pixel_x(lon, zoom) + (256 * scale)
        new_lon = pixel_x_to_lon(pixel_x, zoom)
        pixel_y = lat_to_pixel_y(lat, zoom) + (256 * scale)
        new_lat = pixel_y_to_lat(pixel_y, zoom)
        diagonal_lat_lon.append((new_lat, new_lon))
        lat, lon = new_lat, new_lon
    return diagonal_lat_lon

def get_points(roi, zoom, scale):
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([coord[0] for coord in bounds])
    lats = sorted([coord[1] for coord in bounds])
    starting_point = (lats[-1], lons[0])
    min_pixel = [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom)]
    max_pixel = [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
    iterations = math.ceil(max(abs(min_pixel[0] - max_pixel[0]), abs(min_pixel[1] - max_pixel[1])) / (256 * scale))
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(f"Generated {len(points)} points")
    for point in points:
        bottom_right = lat_lon_from_pixel(point[0], point[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(point[1], point[0]), (bottom_right[1], bottom_right[0])])
        if roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo():
            intersect_list.append(point)
    return intersect_list

In [95]:
top_left = [23.531653742232088, 85.82344897858329]  
bottom_right = [23.64099145921938, 85.99871524445243]
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
roi = ee.FeatureCollection([ee.Feature(rectangle)])

print("Area of the Rectangle: ", rectangle.area().getInfo()/1e6, "sq km")
points = get_points(roi, zoom, 16)
print(f"Running for {len(points)} points...")

#roi = ee.FeatureCollection("projects/df-project-iit/assets/core-stack/tamil_nadu/theni/periyakulam/filtered_mws_theni_periyakulam_uid").filter(ee.Filter.stringContains("uid", "2_25099"))    


Area of the Rectangle:  217.14611603820478 sq km
Generated 5 points
Running for 3 points...


In [96]:
for index, point in enumerate(points):
    output_dir = os.path.join(image_dir, str(index))
    download(point, output_dir, zoom, 16)

Downloaded image 001/289
Downloaded image 002/289
Downloaded image 003/289
Downloaded image 004/289
Downloaded image 005/289
Downloaded image 006/289
Downloaded image 007/289
Downloaded image 008/289
Downloaded image 009/289
Downloaded image 010/289
Downloaded image 011/289
Downloaded image 012/289
Downloaded image 013/289
Downloaded image 014/289
Downloaded image 015/289
Downloaded image 016/289
Downloaded image 017/289
Downloaded image 018/289
Downloaded image 019/289
Downloaded image 020/289
Downloaded image 021/289
Downloaded image 022/289
Downloaded image 023/289
Downloaded image 024/289
Downloaded image 025/289
Downloaded image 026/289
Downloaded image 027/289
Downloaded image 028/289
Downloaded image 029/289
Downloaded image 030/289
Downloaded image 031/289
Downloaded image 032/289
Downloaded image 033/289
Downloaded image 034/289
Downloaded image 035/289
Downloaded image 036/289
Downloaded image 037/289
Downloaded image 038/289
Downloaded image 039/289
Downloaded image 040/289


# SAVE PREDICTIONS IN CSV

In [76]:
pwd

'/home/core-stack/Documents/Ponds'

In [77]:
conf_thresholds = {
    'Dry': 0.75,
    'Wet': 0.6
}

class_names = ['Dry', 'Wet']


class_abbreviations = {'Dry': 'D', 'Wet': 'W'}

In [78]:
my_new_model = YOLO(model_path)

In [79]:
def get_entropy(img, mask):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)  # Convert to grayscale
    mask[mask > 0] = 1  # Ensure mask is binary
    if mask.shape[:2] != img_gray.shape:
        print("Mask shape does not match image shape")
        return 0

    # Normalize the grayscale image to [0, 1]
    img_gray = (img_gray - img_gray.min()) / (img_gray.max() - img_gray.min())

    # Convert to uint8 after normalization
    img_gray = img_as_ubyte(img_gray)

    ent = entropy(img_gray.copy(), disk(5), mask=mask)
    ent = ent[ent > 5.2]
    if np.sum(mask) > 0:
        ent = np.sum(ent) / np.sum(mask)  # Average entropy based on the mask area
    else:
        ent = 0
    return ent

In [80]:
def process_image(image_path, conf_thresholds):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return None, None, None, None, None

    results = my_new_model.predict(img)

    polygons, pred_classes, conf_scores, entropies = [], [], [], []
    if results[0].masks is not None:
        for i, (polygon, cls, conf) in enumerate(zip(results[0].masks.xy, results[0].boxes.cls.cpu().numpy(), results[0].boxes.conf.cpu().numpy())):
            class_name = class_names[int(cls)]
            if conf >= conf_thresholds[class_name]:
                polygons.append(polygon)
                pred_classes.append(class_name)
                conf_scores.append(conf)
                if class_name == 'Wet':
                    predicted_mask = results[0].masks.data.cpu().numpy()[i].astype(np.uint8)
                    entropies.append(get_entropy(img, predicted_mask))
                else:
                    entropies.append(None)
    return image_path, len(polygons), polygons, pred_classes, conf_scores, entropies

def extract_xtile_ytile_from_tile_name(tile_name):
    try:
        parts = tile_name.replace('.tif', '').split('_')
        if len(parts) == 4:
            xtile = int(parts[2])
            ytile = int(parts[3])
            return xtile, ytile
        else:
            raise ValueError("Tile name does not contain valid coordinates")
    except Exception as e:
        raise ValueError(f"Invalid tile name {tile_name}: {e}")

def tile_corners_to_latlon(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad_nw = math.atan(math.sinh(math.pi * (1 - 2 * (ytile / n))))
    lat_deg_nw = math.degrees(lat_rad_nw)

    lat_rad_se = math.atan(math.sinh(math.pi * (1 - 2 * ((ytile + 1) / n))))
    lat_deg_se = math.degrees(lat_rad_se)

    lat_deg_nw = max(min(lat_deg_nw, 85.0511), -85.0511)
    lat_deg_se = max(min(lat_deg_se, 85.0511), -85.0511)

    top_left = (lat_deg_nw, lon_deg)
    top_right = (lat_deg_nw, lon_deg + (360.0 / n))
    bottom_right = (lat_deg_se, lon_deg + (360.0 / n))
    bottom_left = (lat_deg_se, lon_deg)

    return top_left, top_right, bottom_left, bottom_right

def calculate_tile_center(top_left, top_right, bottom_left, bottom_right):
    center_lat = (top_left[0] + bottom_left[0]) / 2
    center_lon = (top_left[1] + top_right[1]) / 2
    return (center_lat, center_lon)


In [82]:
image_data = []
max_vertices = 0

# Traverse subfolders like TRY/0, TRY/1, ...
for subfolder in sorted(os.listdir(image_dir)):
    subfolder_path = os.path.join(image_dir, subfolder)
    chunk_dir = os.path.join(subfolder_path, "chunks")
    tile_map_path = os.path.join(subfolder_path, "tile_mapping.csv")

    if not os.path.isdir(chunk_dir) or not os.path.exists(tile_map_path):
        continue

    # Load mapping: {chunk_0_0.tif: tile_17_96446_56783.tif}
    tile_mapping = {}
    with open(tile_map_path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            chunk_name, tile_name = line.strip().split(',')
            tile_mapping[chunk_name] = tile_name

    # Process each chunk image
    image_files = [os.path.join(chunk_dir, f) for f in os.listdir(chunk_dir) if f.endswith(".tif")]

    for image_path in image_files:
        chunk_name = os.path.basename(image_path)
        if chunk_name not in tile_mapping:
            print(f"Warning: {chunk_name} not in mapping. Skipping.")
            continue

        tile_name = tile_mapping[chunk_name]
        _, _, polygons, pred_classes, conf_scores, entropies = process_image(image_path, conf_thresholds)

        if polygons:
            max_vertices = max(max_vertices, max(len(polygon) for polygon in polygons))

        image_data.append({
            'tile_name': tile_name,
            'polygons': polygons,
            'pred_classes': pred_classes,
            'entropies': entropies
        })

# ===================== SAVE TO CSV =====================
coordinate_headers = []
for i in range(1, max_vertices + 1):
    coordinate_headers.extend([f"X_{i}", f"Y_{i}"])

header = ["Tile Name", "Predicted Class", "Center Latitude", "Center Longitude",
          "Top Left Latitude", "Top Left Longitude", "Top Right Latitude", "Top Right Longitude",
          "Bottom Left Latitude", "Bottom Left Longitude", "Bottom Right Latitude", "Bottom Right Longitude"] + coordinate_headers

start_time = time.time()
with open(csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(header)

    for data in image_data:
        tile_name = data['tile_name']
        polygons = data['polygons']
        pred_classes = data['pred_classes']
        entropies = data['entropies']

        xtile, ytile = extract_xtile_ytile_from_tile_name(tile_name)
        top_left, top_right, bottom_left, bottom_right = tile_corners_to_latlon(xtile, ytile, zoom)
        latitude, longitude = calculate_tile_center(top_left, top_right, bottom_left, bottom_right)

        for pred_class, polygon, entropy_value in zip(pred_classes, polygons, entropies):
            if entropy_value is None or entropy_value > entropy_threshold:
                print(f"Skipped {pred_class} due to high entropy: {entropy_value if entropy_value is not None else 'None'}")
                continue

            row = [tile_name, pred_class, latitude, longitude, top_left[0], top_left[1],
                   top_right[0], top_right[1], bottom_left[0], bottom_left[1], bottom_right[0], bottom_right[1]]

            flat_polygon = [coord for point in polygon for coord in point]
            flat_polygon += [None] * (2 * max_vertices - len(flat_polygon))
            row.extend(flat_polygon)
            csvwriter.writerow(row)

end_time = time.time()
print(f"CSV file '{csv_file}' saved successfully.")
print(f"Time taken: {end_time - start_time:.2f} seconds.")


0: 256x256 (no detections), 8.9ms
Speed: 0.4ms preprocess, 8.9ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 9.8ms
Speed: 0.5ms preprocess, 9.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Wet, 9.7ms
Speed: 0.4ms preprocess, 9.7ms inference, 2.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dry, 1 Wet, 9.1ms
Speed: 0.4ms preprocess, 9.1ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Wet, 9.5ms
Speed: 0.5ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dry, 9.6ms
Speed: 0.5ms preprocess, 9.6ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Wet, 9.5ms
Speed: 0.5ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 11.0ms
Speed: 0.5ms preprocess, 11.0ms inference, 0.3ms postprocess per image at shape (1, 3, 256

# Add Buffer to combine nearby predicted objects

In [83]:
pwd

'/home/core-stack/Documents/Ponds'

In [84]:
EARTH_CIRCUMFERENCE_DEGREES = 360  # degrees

In [85]:
df = pd.read_csv(csv_file)
df.rename(columns={'Predicted Class': 'Class'}, inplace=True) 

In [86]:
def pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, img_width, img_height):
    lon_range = lon_bottom_right - lon_top_left
    lat_range = lat_top_left - lat_bottom_right  # Ensure correct sign

    lon = lon_top_left + (x / img_width) * lon_range
    
    # Ensure latitude decreases as 'y' increases (accounting for top-left origin)
    lat = lat_top_left - (y / img_height) * lat_range

    return lon, lat


# Initialize an empty list for GeoJSON features
geojson_features = []

# Iterate through each row
for _, row in df.iterrows():
    lat_top_left = row['Top Left Latitude']
    lon_top_left = row['Top Left Longitude']
    lat_bottom_right = row['Bottom Right Latitude']
    lon_bottom_right = row['Bottom Right Longitude']

    tile_width, tile_height = 256, 256

    object_coords = []

    # Iterate over all possible coordinate columns dynamically
    i = 1
    while True:
        x_col = f'X_{i}'
        y_col = f'Y_{i}'

        if x_col not in row or y_col not in row:
            break  # Stop if columns don't exist

        x = row[x_col]
        y = row[y_col]

        if pd.notna(x) and pd.notna(y):
            lon, lat = pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, tile_width, tile_height)
            if np.isfinite(lon) and np.isfinite(lat):
                object_coords.append((lon, lat))
        else:
            break  # Stop when NaN values appear

        i += 1  # Move to the next set of coordinates

    # Ensure the polygon has at least 3 points before adding
    if len(object_coords) >= 3:
        object_coords.append(object_coords[0])  # Close the polygon
        polygon_geometry = Polygon(object_coords)
        feature = {
            "type": "Feature",
            "geometry": polygon_geometry,
            "properties": {
                "Class": row['Class']
            }
        }
        geojson_features.append(feature)

# Convert to GeoDataFrame
gdf_final = gpd.GeoDataFrame(
    [feature['properties'] for feature in geojson_features],
    geometry=[feature['geometry'] for feature in geojson_features],
    crs="EPSG:4326"
)

# Merge overlapping geometries


# Merge overlapping geometries
buffer_distance = 0.0005  
gdf_final['Buffered'] = gdf_final.geometry.buffer(buffer_distance)
combined_polygons = unary_union(gdf_final['Buffered'])
combined_polygons = combined_polygons.buffer(-buffer_distance)

# Convert back to GeoDataFrame
gdf_combined = gpd.GeoDataFrame(geometry=[combined_polygons], crs="EPSG:4326")

# Ensure the output folder exists
output_folder = 'Shapefile_Output'
os.makedirs(output_folder, exist_ok=True)

# Save the final shapefile in the 'Shapefile_Output' folder
shapefile_path = os.path.join(output_folder, f"{csv_basename}_COMBINED_GEOMETRY.shp")
gdf_combined.to_file(shapefile_path)

# Create the ZIP file in the 'Shapefile_Output' folder
zip_filename = os.path.join(output_folder, f"{csv_basename}_COMBINED_GEOMETRY.zip")
shapefile_files = glob.glob(os.path.join(output_folder, f"{csv_basename}_COMBINED_GEOMETRY.*"))  

# Exclude the .csv file from the list of files to be zipped
shapefile_files = [file for file in shapefile_files if not file.endswith('.csv')]

# Add the shapefile files to the zip archive
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in shapefile_files:
        zipf.write(file, os.path.basename(file))

print(f"Created ZIP file: {zip_filename}")

# Delete the original shapefile files after zipping, but keep the .csv file
for file in shapefile_files:
    os.remove(file)
    print(f"Deleted: {file}")

print("All shapefile components have been deleted after zipping, except for the .csv file.")

Created ZIP file: Shapefile_Output/TRY_COMBINED_GEOMETRY.zip
Deleted: Shapefile_Output/TRY_COMBINED_GEOMETRY.dbf
Deleted: Shapefile_Output/TRY_COMBINED_GEOMETRY.prj
Deleted: Shapefile_Output/TRY_COMBINED_GEOMETRY.cpg
Deleted: Shapefile_Output/TRY_COMBINED_GEOMETRY.shx
Deleted: Shapefile_Output/TRY_COMBINED_GEOMETRY.shp
All shapefile components have been deleted after zipping, except for the .csv file.


/tmp/ipykernel_1319839/4198362238.py:73: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_final['Buffered'] = gdf_final.geometry.buffer(buffer_distance)
